In [1]:
import random
from faker import Factory
import pinyin
import psycopg2, psycopg2.extras

In [2]:
conn = psycopg2.connect(dbname='DBS2017_new', user='postgres')
cur = conn.cursor()

In [3]:
print('Generating accounts...', end=' ')

fake = Factory.create()

def get_username():
    first = fake.first_name()
    last = fake.last_name()
    return str.lower(
        first[:random.randint(1, len(first))] +
        last[:random.randint(1, len(last))]
    )

usernames = [get_username() for _ in range(100)]
usernames_gen = (
    (random.choice(usernames) + ''.join([random.choice('1234567890') for _ in range(random.randint(2, 4))]),)
    for _ in range(500000)
)

psycopg2.extras.execute_batch(
    cur, 'insert into account (username) values (%s) '
    'on conflict (username) do nothing',
    usernames_gen, 10000
)

print('Done.')

Generating accounts... Done.


In [4]:
print('Adding countries...', end=' ')

# Source: https://gist.github.com/kalinchernev/486393efcca01623b18d
with open('countries.txt') as file:
    cur.executemany(
        'insert into country (name) values (%s)',
        [(line.strip(),) for line in file]
    )

print('Done.')

Adding countries... Done.


In [5]:
print('Adding regions...', end=' ')

regions = {
    'China': ['Anhui', 'Beijing', 'Chongqing', 'Fujian', 'Gansu', 'Guangdong', 'Guangxi', 'Guizhou', 'Hebei', 'Henan', 'Hubei', 'Hunan', 'Inner Mongolia', 'Jiangsu', 'Jiangxi', 'Jilin', 'Liaoning', 'Ningxia', 'Shaanxi', 'Shandong', 'Shanghai', 'Shanxi', 'Sichuan', 'Tianjin', 'Yunnan', 'Zhejiang'],
    'Taiwan': ['Changhua', 'Chiayi', 'Hsinchu', 'Hualien', 'Kinmen', 'Lienchiang', 'Miaoli', 'Nantou', 'Penghu', 'Pingtung', 'Taitung', 'Taoyuan', 'Yilan', 'Yunlin'],
    'Japan': ['Aichi', 'Akita', 'Aomori', 'Chiba', 'Ehime', 'Fukui', 'Fukuoka', 'Fukushima', 'Gifu', 'Gunma', 'Hiroshima', 'Hokkaido', 'Hyogo', 'Ibaraki', 'Ishikawa', 'Iwate', 'Kagawa', 'Kagoshima', 'Kanagawa', 'Kochi', 'Kumamoto', 'Kyoto', 'Mie', 'Miyagi', 'Miyazaki', 'Nagano', 'Nagasaki', 'Nara', 'Niigata', 'Oita', 'Okayama', 'Okinawa', 'Osaka', 'Saga', 'Saitama', 'Shiga', 'Shimane', 'Shizuoka', 'Tochigi', 'Tokushima', 'Tokyo', 'Tottori', 'Toyama', 'Wakayama', 'Yamagata', 'Yamaguchi', 'Yamanashi'],
    'India': ['Andhra Pradesh', 'Arunachal Pradesh', 'Assam', 'Bihar', 'Chattisgarh', 'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh', 'Jharkhand', 'Jammu And Kashmir', 'Karnataka', 'Kerala', 'Madhya Pradesh', 'Manipur', 'Maharastra', 'Meghalaya', 'Mizoram', 'Nagaland', 'Odisha', 'Punjab', 'Rajasthan', 'Sikkim', 'Tamil Nadu', 'Telangana', 'Tripura', 'Uttarakhand', 'Uttar Pradesh', 'West Bengal']
}

cur.executemany(
    'insert into origin (name, country_id) values '
    '(%s, (select id from country where name like %s))',
    [(origin, country) for country, region in regions.items() for origin in region]
)

print('Done.')

Adding regions... Done.


In [6]:
print('Adding categories...', end=' ')

categories = ['White', 'Green', 'Oolong', 'Black', 'Yellow', 'Dark']

cur.executemany(
    'insert into category (name) values (%s)',
    [(category,) for category in categories]
)

print('Done.')

Adding categories... Done.


In [7]:
print('Generating varieties...', end=' ')

cur.execute('select id from category')
category_ids = cur.fetchall()

cur.execute('alter table variety disable trigger user')
psycopg2.extras.execute_batch(
    cur, 'insert into variety (name, category_id) values (%s, %s)'
    'on conflict do nothing',
    ((str.capitalize(''.join(random.sample(list(pinyin.pinyin.pinyin_dict.values()), random.randint(2, 4)))),
     random.choice(category_ids)[0]) for _ in range(200000)), 10000
)
cur.execute('alter table variety enable trigger user')

print('Done.')

Generating varieties... Done.


In [8]:
print('Generating vendors...', end=' ')

fake = Factory.create('en_GB')

cur.execute('select id from country')
country_ids = cur.fetchall()

cur.execute('alter table vendor disable trigger user')
psycopg2.extras.execute_batch(
    cur, 'insert into vendor (name, country_id) values (%s, %s)'
    'on conflict do nothing',
    ((fake.company(), random.choice(country_ids)[0]) for _ in range(50000)), 10000
)
cur.execute('alter table vendor enable trigger user')

print('Done.')

Generating vendors... Done.


In [9]:
print('Generating teas...', end=' ')

prefixes = ['First flush ', 'Second flush ', 'Spring ', 'Summer ', 'Autumn ', 'Winter ', 'Roasted ', 'Aged ', 'High mountain ', 'Imperial ', 'Ceremonial ', 'GABA ', 'Fresh ', 'Light roast ', 'Medium roast ', 'High fired ', 'Charcoal roasted ', 'Jasmine ', 'Classic ', 'Traditional ', 'Certified Organic ', 'Early Spring ', 'Late ']
suffixes = [' tea', ' Cha', ' Jinzhen', 'Gushu', ' Gongfu', ' Maofeng', ' Premium', ' OP', ' FGTOP', ' BIO', ' Organic', ' Superior', ' Top Grade', ' AAA Grade', ' Competition Grade', ' Special Grade', ' Cake', ' Brick', ' Loose Leaf', ' Silver Needle'] + [' ' + str(y) for y in range(1980, 2017)]

cur.execute('select id from origin')
origin_ids = cur.fetchall()

cur.execute('select id from vendor')
vendor_ids = cur.fetchall()

cur.execute('select id from variety')
variety_ids = cur.fetchall()

cur.execute('alter table tea disable trigger user')
psycopg2.extras.execute_batch(
    cur, 'insert into tea (variety_id, name, origin_id, vendor_id) '
    'select id, %s || name || %s || %s, %s, %s '
    'from variety where id = %s',
    ((random.choice(prefixes), random.choice(suffixes), random.choice(suffixes),
      random.choice(origin_ids)[0], random.choice(vendor_ids)[0], random.choice(variety_ids)[0]
     ) for _ in range(1500000)), 10000
)
cur.execute('alter table tea enable trigger user')

print('Done.')

Generating teas... Done.


In [10]:
print('Generating purchases...', end=' ')

cur.execute('select id from account')
account_ids = cur.fetchall()

cur.execute('select id from tea')
tea_ids = cur.fetchall()

def get_selection():
    for account_id in account_ids:
        tea_sample = random.sample(tea_ids, 8)
        for _ in range(random.randint(0, 6)):
            yield (account_id[0], random.choice(tea_sample)[0])

cur.execute('alter table purchase disable trigger user')
psycopg2.extras.execute_batch(
    cur, 'insert into purchase (purchased, active, account_id, tea_id) values '
    '(date \'2017-01-01\' - random() * interval \'1 year\', random() < 0.8, %s, %s)',
    get_selection(), 10000
)
cur.execute('alter table purchase enable trigger user')

print('Done.')

Generating purchases... Done.


In [11]:
print('Generating ratings...', end=' ')

cur.execute('alter table variety disable trigger user')
psycopg2.extras.execute_batch(
    cur, 'insert into rating (score, account_id, tea_id) values (%(s)s, %(a)s, %(t)s) '
    'on conflict (account_id, tea_id) do update set score = %(s)s, updated = now()',
    ({'s': random.randint(0, 100), 'a': random.choice(account_ids)[0], 't': random.choice(tea_ids)[0]}
     for _ in range(200000)), 10000
)
cur.execute('alter table variety enable trigger user')

print('Done.')

Generating ratings... Done.


In [12]:
print('Adding flavors...', end=' ')

flavors = ['Banana', 'Amylacetate', 'Brown', 'Candy-like', 'Cooked', 'Creamy', 'Diffusive', 'Estery', 'Floral, Geranium', 'Fresh, Sulfidic', 'Green', 'Mashed', 'Peely', 'Pulpy', 'Ripe', 'Rotten', 'Starchy', 'Astringent', 'Earthy', 'Fermented', 'Fresh', 'Green', 'Jammy', 'Juicy', 'Musty', 'Perfumey', 'Seedy', 'Sweet', 'Woody', 'Berry', 'Cooling', 'Floral, Linalool', 'Fresh', 'Fruity', 'Grape', 'Green', 'Green, Stemmy', 'Jammy', 'Musky', 'Perfumey', 'Ripe', 'Tart', 'Woody', 'Browned', 'Burnt', 'Butterscotch', 'Buttery', 'Caramelized Sugar', 'Caramellic', 'Cooked', 'Creamy', 'Custard', 'Drying', 'Egg', 'Fruity', 'Honey', 'Malt', 'Maple', 'Milky', 'Molasses', 'Nutty', 'Oxidized', 'Raisin', 'Rich', 'Roasted', 'Smoky', 'Toasted', 'Toffee', 'Vanilla', 'Vanillin', 'Waxy', 'Cherry', 'Benzaldehyde', 'Berry, Strawberry', 'Bitter Almond Nut', 'Burning Bite', 'Candy-like', 'Cinnamon-like', 'Citrus, Citronella', 'Cooked', 'Floral, Rosy', 'Fresh', 'Fruity, Banana', 'Green, Aldehydic', 'Juicy', 'Liqueur Type', 'Maraschino', 'Medicinal', 'Metallic', 'Morello, Dark', 'Mushy', 'Musty', 'Pit Type', 'Red (Amyl Butyrate)', 'Ripe, Bruised', 'Sour', 'Sweet', 'Tart', 'Wild', 'Winey', 'Alkalized', 'Astringent', 'Acidic, Harsh', 'Beany', 'Bitter', 'Black Cocoa', 'Botanical', 'Caramellic', 'Cocoa Powder', 'Coffee, Burnt', 'Dairy, Buttery', 'Dark Chocolate', 'Dirty', 'Dutched', 'Earthy', 'Ethyl Vanillin', 'Fatty', 'Floral', 'Fruity', 'Fudgey', 'Green', 'Hot Chocolate', 'Ice Cream', 'Liqueur', 'Malted Milk Balls', 'Malty', 'Marshmallow', 'Medicinal, Phenolic', 'Milky', 'Musty', 'Nutty', 'Oily', 'Perfumey', 'Pyrazine', 'Resinous', 'Roasted', 'Rosy', 'Rubbery', 'Semi-sweet', 'Skunky', 'Sour', 'Spicy', 'Stale', 'Sulfitic', 'Sweet', 'Tea', 'Tobacco', 'Vanillin', 'White Chocolate', 'Woody', 'Buttery, Estery', 'Candy-like', 'Caramellic, Maltol', 'Cool, Frosty', 'Diffusive', 'Dry', 'Floral, Perfumey', 'Fresh', 'Fruity, Banana', 'Green, Seedless', 'Jammy, Jelly', 'Juicy', 'Leafy', 'Lemon, Citrus', 'Methyl Anthranilate', 'Musty', 'Purple, Concord', 'Ripe', 'Seedy', 'Sour, Acetic Acid', 'Sweet', 'Tart', 'Vanillin Sweet', 'Woody', 'Aldehydic, Decanal', 'Bitter, Naringin', 'Bitter, Orange', 'Chemical, Artificial', 'Citrusy, Peel Oil', 'Dry, Astringent', 'Fresh', 'Green', 'Juicy', 'Peely, Nootkatone', 'Pungent', 'Sulfur', 'Tart, Sour', 'Terpeney', 'Green Apple', 'Acidic', 'Astringent', 'Bitter', 'Buttery', 'Cooked', 'Earthy', 'Estery', 'Fresh Fruit', 'Fruity, Green', 'Green', 'Juicy', 'Leafy', 'Mealy', 'Peely', 'Raw', 'Sour', 'Tart', 'Waxy', 'Acidic', 'Aldehydic, Citral', 'Candy-like', 'Citral', 'Citrus', 'Cooked, Pie Filling', 'Fleshy', 'Floral, Linalool', 'Floral, Rosy, Geraniol', 'Fresh', 'Juicy', 'Lemongrass, Green', 'Lime', 'Over-ripe', 'Peely, Rindy', 'Seedy', 'Sour, Tart', 'Sulfide', 'Sweet', 'Terpeney, Oxidized, Stale', 'Waxy', 'Woody', 'Fresh, Green', 'Acidic', 'Juicy', 'Fleshy', 'Lemon, Citral', 'Sweet', 'Piney', 'Dry, Medicinal', 'Oily', 'Camporaceous', 'Citrusy', 'Terpeney, Oxidized', 'Floral, Terpineol', 'Peely', 'Acidic', 'Alfalfa', 'Anise', 'Astringent, Drying', 'Balanced', 'Botanical', 'Bready', 'Brown Sugar', 'Burnt, Roasted', 'Butter', 'Butterscotch', 'Buttery, Creamy', 'Caramellic', 'Celery-like', 'Chocolate', 'Coffee, Roasted', 'Corn Syrup', 'Cotton Candy', 'Floral', 'Foenugreek', 'Fruity', 'Green', 'Hay-like', 'Herbal', 'Honey', 'Horehound', 'Licorice', 'Malty', 'Maple Sap', 'Marshmallow', 'Metallic', 'Molasses', 'Musty, Mossy', 'Nutty', 'Perfumey', 'Pruney', 'Raisin', 'Rummy', 'Salty', 'Smoky', 'Sorghum', 'Tinny', 'Tobacco', 'Vanilla', 'Vanillin', 'Weedy', 'Yeasty', 'Acidic', 'Aldehydic, Fatty', 'Bitter Orange', 'Citrusy', 'Creamy, Cardboardy', 'Estery, Chemical', 'Fleshy', 'Floral, Waxy', 'Fresh, Green', 'Fruity', 'Juicy', 'Mandarin, Tangerine', 'Oily', 'Peely', 'Peely, Rindy', 'Seed Note', 'Sweet', 'Tart, Sour', 'Terpeney, Oxidized', 'Woody', 'Acidic', 'Anisic', 'Aromatic, Heavy', 'Camphoraceous', 'Cedarwood', 'Chemical', 'Clean', 'Cool, Cooling', 'Crushed Leaf', 'Drying', 'Earthy, Musty', 'Ethereal', 'Eucalyptus', 'Fragrant', 'Fresh', 'Fruity, Pear-like', 'Grassy', 'Green', 'Herbal', 'Ketonic', 'Medicinal', 'Mentholic', 'Metallic', 'Minty', 'Oily', 'Peroxide', 'Pine Oil', 'Sharp, Pungent', 'Solvent', 'Sour, Vinegar', 'Sweet', 'Vanilla', 'Weedy', 'Woody, Resinous', 'Berry', 'Butter', 'Cherry', 'Floral', 'Fresh', 'Fruity', 'Green', 'Jammy', 'Perfumey', 'Ripe', 'Rosy', 'Seedy', 'Soapy', 'Sulfuraceous', 'Tart', 'Blossom', 'Buttery', 'Candy-like', 'Chemical', 'Cooked', 'Dry', 'Estery', 'Floral', 'Fresh', 'Fruity', 'Green', 'Juicy', 'Mealy', 'Musty', 'Peely', 'Pulpy', 'Red', 'Ripe', 'Sweet', 'Winey', 'Balsamic', 'Camphoraceous', 'Clean', 'Clove', 'Crushed Leaf', 'Diffusive', 'Drying', 'Ethereal', 'Fragrant', 'Fresh', 'Fruity', 'Green', 'Heavy', 'Herbal', 'Ketonic, Carvone', 'Medicinal', 'Minty', 'Oily', 'Smooth', 'Solvent', 'Spicy', 'Sulfidic, Musty', 'Sweet', 'Terpeney', 'Warm', 'Weedy', 'Alliaceous (Onion-like)', 'Burnt', 'Buttery', 'Candy-like', 'Cooked', 'Cotton Candy', 'Creamy', 'Floral, Rosy', 'Fresh', 'Fruity, Berry Maltol', 'Green', 'Jammy', 'Ripe', 'Seedy', 'Sweet', 'Acidic', 'Astringent', 'Baby Powder', 'Balanced', 'Beany', 'Bitter', 'Caramellic', 'Cocoa, Chocolate', 'Cotton Candy, Marshmallow', 'Creamy, Dairy, Fatty, Rich-tasting', 'Fig-like', 'Floral', 'Fruity', 'Harsh', 'Hay-like', 'Heliotrope (Floral)', 'Honey-like', 'Maple-like', 'Nutty', 'Perfumey', 'Prune-like', 'Raisin-like', 'Roasted', 'Rum-like', 'Sharp', 'Smoky', 'Smooth', 'Spicy', 'Sweet', 'Tobacco', 'Unbalanced', 'Winey', 'Woody', 'Candy-like', 'Cantaloupe-like Chemical,', 'Melonal Cherry-like', 'Estery', 'Fresh, Green Juicy', 'Melony', 'Overripe, Rotten Pickled', 'Pulpy', 'Rindy', 'Seedy', 'Sweet, Sugary', 'Aromatic', 'Bark, Birch-like Burning', 'Camphoraceous Chemical', 'Cool', 'Fermented', 'Floral', 'Fragrant', 'Fruity', 'Grassy', 'Green, Rooty Herbal', 'Medicinal', 'Mentholic', 'Methyl Salicylate Minty', 'Oily', 'Pungent, Strong Spicy', 'Sweet', 'Warm, Leafy']

cur.executemany(
    'insert into flavor (name) values (%s)',
    ((flavor,) for flavor in flavors)
)

print('Done.')

Adding flavors... Done.


In [13]:
conn.commit()

In [25]:
conn.rollback()

In [14]:
conn.close()